# Luke McMeans - Finance API Homework

**TASK 1: Fetch Basic Stock Data**

In [29]:
# api key
api_key = ""

In [30]:
import pandas as pd
import requests

# get inputted stocks, as many until they decide to quit
idx = 1
print("Input at least 2 stocks (type 'q' to quit)")
stocks = []
cont = True
while(cont):
    stock = input(f"Input Stock {idx}: ")
    idx += 1
    if stock == 'q':
        break
    stocks.append(stock)

# in case no stocks were inputted, give a sample
if not stocks:
    stocks = ['AAPL', 'MSFT', 'MSTR']

Input at least 2 stocks (type 'q' to quit)


In [31]:
# convert to query string
stock_str = ",".join(stocks)
query_string = {'symbols': stock_str}

# fetch data
url = "https://yfapi.net/v6/finance/quote"
headers = {
    'x-api-key': api_key
}
response = requests.request("GET", url, headers=headers, params=query_string)

In [32]:
# convert to dataframe, and translate market price to 
df = pd.DataFrame(response.json()['quoteResponse']['result'])

# function to translate columns to proper dollar strings
def convert_to_dollar(col):
    df[col] = df[col].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A")

convert_to_dollar('regularMarketPrice')

# print statements for all rows
for _, row in df.iterrows():
    ticker = row['symbol']
    name = row['shortName']
    market_price = row['regularMarketPrice']
    
    print(f"Stock Ticker: {ticker}, Company: {name}, Current Market Price: {market_price}")

df

Stock Ticker: AAPL, Company: Apple Inc., Current Market Price: $245.13
Stock Ticker: MSFT, Company: Microsoft Corporation, Current Market Price: $398.22
Stock Ticker: MSTR, Company: MicroStrategy Incorporated, Current Market Price: $251.62


,language,region,quoteType,typeDisp,quoteSourceName,triggerable,customPriceAlertConfidence,currency,regularMarketChangePercent,regularMarketPrice,...,messageBoardId,exchangeTimezoneName,exchangeTimezoneShortName,gmtOffSetMilliseconds,market,esgPopulated,shortName,longName,displayName,symbol
0,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,-0.797249,$245.13,...,finmb_24937,America/New_York,EST,-18000000,us_market,False,Apple Inc.,Apple Inc.,Apple,AAPL
1,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,-1.430693,$398.22,...,finmb_21835,America/New_York,EST,-18000000,us_market,False,Microsoft Corporation,Microsoft Corporation,Microsoft,MSFT
2,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,-11.012877,$251.62,...,finmb_384976,America/New_York,EST,-18000000,us_market,False,MicroStrategy Incorporated,MicroStrategy Incorporated,NaN,MSTR


**TASK 2.1: Fetch Additional Data Using Modules**

Note: The 52WeekHigh, 52WeekLow, and ReturnOnAssests are found in the `summaryDetail` and `financialData` modules, so they are automatically retrieved. Other modules can be selected and turned into it's own dataframe.

In [33]:
# get inputted modules, as many until they decide to quit
idx = 1
print("Input modules you wan't to find (type 'q' to stop)")
print("OPTIONS: assetProfile, fundProfile, defaultKeyStatistics, calendarEvents, \n \
      incomeStatementHistory, incomeStatementHistoryQuarterly, cashflowStatementHistory, \n \
      earnings, earningsHistory, insiderHolders,balanceSheetHistory, cashflowStatementHistory, \n \
      cashflowStatementHistoryQuarterly, insiderTransactions, secFilings, indexTrend, \n \
      sectorTrend, earningsTrend, netSharePurchaseActivity, upgradeDowngradeHistory,\n \
      institutionOwnership, recommendationTrend, balanceSheetHistory, balanceSheetHistoryQuarterly,\n \
      fundOwnership, majorDirectHolders, majorHoldersBreakdown, price, quoteType, esgScores")
modules = ['summaryDetail', 'financialData']
cont = True
while(cont):
    mod = input(f"Input Module {idx}: ")
    idx += 1
    if mod == 'q':
        break
    modules.append(mod)

Input modules you wan't to find (type 'q' to stop)
OPTIONS: assetProfile, fundProfile, defaultKeyStatistics, calendarEvents, 
       incomeStatementHistory, incomeStatementHistoryQuarterly, cashflowStatementHistory, 
       earnings, earningsHistory, insiderHolders,balanceSheetHistory, cashflowStatementHistory, 
       cashflowStatementHistoryQuarterly, insiderTransactions, secFilings, indexTrend, 
       sectorTrend, earningsTrend, netSharePurchaseActivity, upgradeDowngradeHistory,
       institutionOwnership, recommendationTrend, balanceSheetHistory, balanceSheetHistoryQuarterly,
       fundOwnership, majorDirectHolders, majorHoldersBreakdown, price, quoteType, esgScores


In [34]:
# let's use the first inputted stock for reference
stock = stocks[0]

# convert to query string
module_str = ",".join(modules)
query_string = {
    'symbols': stock,
    'modules': module_str
}

# fetch data
url = f"https://yfapi.net/v11/finance/quoteSummary/{stock}"
headers = {
    'x-api-key': api_key
}
response = requests.request("GET", url, headers=headers, params=query_string)

In [35]:
result = response.json()['quoteSummary']['result']

# retrieve dataframes 
stats_dfs = []
for key, value in result[0].items():
    # for all values not summaryDetail and financialData
    if key != 'summaryDetail' and key != 'financialData':
        if isinstance(value, dict):
            df = pd.DataFrame([value])
        else:
            df = pd.DataFrame(value)
    else:
        df = pd.DataFrame(value)
        stats_dfs.append(df)
    # print df
    print(f"\nDataframe for module: {key}")
    print(df)

# print 

# find and print desired stats for this portion
stats_dfs[0] = stats_dfs[0].loc['fmt']
stats_dfs[1] = stats_dfs[1].loc['fmt']
weekHigh = stats_dfs[0]['fiftyTwoWeekHigh']
weekLow = stats_dfs[0]['fiftyTwoWeekLow']
roa = stats_dfs[1]['returnOnAssets']
print(f"\n52 Week High: {weekHigh} \n52 Week Low: {weekLow} \nReturn on Assets: {roa}")


Dataframe for module: summaryDetail
         maxAge priceHint previousClose    open  dayLow dayHigh  \
raw           1         2         247.1   248.0  245.19  249.52   
fmt           1         2        247.10  248.00  245.19  249.52   
longFmt       1         2           NaN     NaN     NaN     NaN   

        regularMarketPreviousClose regularMarketOpen regularMarketDayLow  \
raw                          247.1             248.0              245.19   
fmt                         247.10            248.00              245.19   
longFmt                        NaN               NaN                 NaN   

        regularMarketDayHigh  ... toCurrency lastMarket coinMarketCapLink  \
raw                   249.52  ...       None       None              None   
fmt                   249.52  ...       None       None              None   
longFmt                  NaN  ...       None       None              None   

        volume24Hr volumeAllCurrencies circulatingSupply algorithm maxSupply  \


**TASK 2.2: Fetch Additional Data Using Modules**

In [36]:
# get trending/popular stocks
url = "https://yfapi.net/v1/finance/trending/US"  # US market trending stocks
headers = {
    'x-api-key': api_key
}
response = requests.get(url, headers=headers)
trending_data = response.json()

# turn the trending stocks into array
stocks = []
for quote in trending_data['finance']['result'][0]['quotes']:
    stocks.append(quote['symbol'])

In [37]:
# convert to query string
stock_str = ",".join(stocks)
query_string = {'symbols': stock_str}

# fetch data
url = "https://yfapi.net/v6/finance/quote"
headers = {
    'x-api-key': api_key
}
response = requests.request("GET", url, headers=headers, params=query_string)

In [38]:
# convert to dataframe, and translate market price to 
df = pd.DataFrame(response.json()['quoteResponse']['result'])

# convert columns to dolar value
convert_to_dollar('regularMarketPrice')
convert_to_dollar('fiftyTwoWeekHigh')
convert_to_dollar('fiftyTwoWeekLow')

print("Trending Stocks:")
df[['symbol', 'shortName', 'regularMarketPrice', 'fiftyTwoWeekHigh', 'fiftyTwoWeekLow']]

Trending Stocks:


,symbol,shortName,regularMarketPrice,fiftyTwoWeekHigh,fiftyTwoWeekLow
0,BTC-USD,Bitcoin USD,"$86,632.53","$109,114.88","$49,121.24"
1,HD,"Home Depot, Inc. (The)",$394.11,$439.37,$323.77
2,HIMS,"Hims & Hers Health, Inc.",$37.00,$72.98,$11.20
3,MSTR,MicroStrategy Incorporated,$252.05,$543.00,$84.18
4,PYPL,"PayPal Holdings, Inc.",$72.91,$93.66,$56.97
5,ONVO,"Organovo Holdings, Inc.",$1.33,$1.74,$0.32
6,O,Realty Income Corporation,$55.37,$64.88,$50.65
7,BMO.TO,BANK OF MONTREAL,$149.07,$151.08,$109.02
8,LI,Li Auto Inc.,$29.48,$46.44,$17.44
9,DOCN,"DigitalOcean Holdings, Inc.",$41.35,$47.02,$26.63
